# Kipindi cha 5 – Mpangaji wa Wakala Wengi

Inaonyesha mchakato rahisi wa mawakala wawili (Mtafiti -> Mhariri) kwa kutumia Foundry Local.


### Maelezo: Usakinishaji wa Vitegemezi
Inasakinisha `foundry-local-sdk` na `openai` zinazohitajika kwa ufikiaji wa modeli za ndani na kukamilisha mazungumzo. Inaweza kurudiwa bila athari mbaya.


# Hali
Inatekeleza muundo wa msingi wa wakala wawili:
- **Wakala Mtafiti** hukusanya vidokezo vya ukweli kwa ufupi
- **Wakala Mhariri** huandika upya kwa uwazi wa kiutendaji

Inaonyesha kumbukumbu ya pamoja kwa kila wakala, kupitisha kwa mfululizo matokeo ya kati, na kazi rahisi ya bomba. Inaweza kupanuliwa kwa majukumu zaidi (mfano, Mkosoaji, Mhakiki) au matawi sambamba.

**Vigezo vya Mazingira:**
- `FOUNDRY_LOCAL_ALIAS` - Mfano wa chaguo-msingi wa kutumia (chaguo-msingi: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Mfano wa wakala wa msingi (unazidi ALIAS)
- `AGENT_MODEL_EDITOR` - Mfano wa wakala mhariri (chaguo-msingi ni wa msingi)

**Marejeleo ya SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Jinsi inavyofanya kazi:**
1. **FoundryLocalManager** huanzisha huduma ya Foundry Local kiotomatiki
2. Inapakua na kupakia mfano uliotajwa (au hutumia toleo lililohifadhiwa)
3. Inatoa sehemu ya mwisho inayolingana na OpenAI kwa mwingiliano
4. Kila wakala anaweza kutumia mfano tofauti kwa kazi maalum
5. Mantiki ya kujirudia iliyojengwa ndani hushughulikia kushindwa kwa muda kwa urahisi

**Vipengele Muhimu:**
- ✅ Ugunduzi wa huduma na uanzishaji kiotomatiki
- ✅ Usimamizi wa mzunguko wa maisha wa mfano (kupakua, kuhifadhi, kupakia)
- ✅ Utangamano wa SDK ya OpenAI kwa API inayofahamika
- ✅ Msaada wa mifano mingi kwa utaalamu wa wakala
- ✅ Ushughulikiaji wa makosa thabiti na mantiki ya kujirudia
- ✅ Utoaji wa maelezo wa ndani (hakuna API ya wingu inayohitajika)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Maelezo: Uingizaji Muhimu & Uandishi
Inatambulisha dataclasses kwa ajili ya kuhifadhi ujumbe wa wakala na vidokezo vya uandishi kwa uwazi. Inaingiza Foundry Local manager + mteja wa OpenAI kwa hatua za wakala zinazofuata.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Maelezo: Uanzishaji wa Modeli (Mfumo wa SDK)
Inatumia Foundry Local Python SDK kwa usimamizi thabiti wa modeli:
- **FoundryLocalManager(alias)** - Huanzisha huduma kiotomatiki na kupakia modeli kwa kutumia alias
- **get_model_info(alias)** - Hutafsiri alias kuwa ID halisi ya modeli
- **manager.endpoint** - Hutoa endpoint ya huduma kwa mteja wa OpenAI
- **manager.api_key** - Hutoa API key (hiari kwa matumizi ya ndani)
- Inasaidia modeli tofauti kwa mawakala tofauti (kama wakuu dhidi ya wahariri)
- Ina mantiki ya kujirudia iliyojengwa ndani na backoff ya kielelezo kwa uimara
- Uhakikisho wa muunganisho ili kuhakikisha huduma iko tayari

**Mfumo Muhimu wa SDK:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Usimamizi wa Mzunguko wa Maisha:**
- Wasimamizi huhifadhiwa kimataifa kwa usafi sahihi
- Kila wakala anaweza kutumia modeli tofauti kwa utaalamu
- Ugunduzi wa huduma kiotomatiki na usimamizi wa muunganisho
- Kujirudia kwa neema na backoff ya kielelezo wakati wa kushindwa

Hii inahakikisha uanzishaji sahihi kabla ya upangaji wa mawakala kuanza.

**Rejea:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Maelezo: Madarasa ya Agent & Memory
Hufafanua `AgentMsg` nyepesi kwa ajili ya maingizo ya kumbukumbu na `Agent` inayojumuisha:
- **Jukumu la Mfumo** - Nafasi ya wakala na maelekezo yake
- **Historia ya Ujumbe** - Hudumisha muktadha wa mazungumzo
- **Njia ya act()** - Hufanya vitendo kwa kushughulikia makosa ipasavyo

Wakala anaweza kutumia mifano tofauti (ya msingi dhidi ya mhariri) na huhifadhi muktadha wa pekee kwa kila wakala. Muundo huu huwezesha:
- Uendelevu wa kumbukumbu kati ya vitendo
- Uteuzi rahisi wa mfano kwa kila wakala
- Kutengwa kwa makosa na urejeshaji
- Urahisi wa kuunganisha na kupanga shughuli


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Maelezo: Njia ya Mchakato Iliyoratibiwa
Inaunda mawakala wawili maalum:
- **Mtafiti**: Hutumia mfano wa msingi, hukusanya taarifa za ukweli
- **Mhariri**: Anaweza kutumia mfano tofauti (ikiwa umewekwa), husahihisha na kuandika upya

Kazi ya `pipeline`:
1. Mtafiti hukusanya taarifa ghafi
2. Mhariri husahihisha na kuibadilisha kuwa matokeo yaliyo tayari kwa utekelezaji
3. Inarudisha matokeo ya kati na ya mwisho

Mfumo huu unaruhusu:
- Utaalamu wa mfano (miundo tofauti kwa majukumu tofauti)
- Kuboresha ubora kupitia usindikaji wa hatua nyingi
- Ufuatiliaji wa mabadiliko ya taarifa
- Urahisi wa kuongeza mawakala zaidi au usindikaji sambamba


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Maelezo: Utekelezaji wa Njia ya Uendeshaji & Matokeo
Inatekeleza njia ya mawakala wengi kwenye swali lenye mada ya uzingatiaji + ucheleweshaji ili kuonyesha:
- Mabadiliko ya taarifa kwa hatua nyingi
- Utaalamu wa mawakala na ushirikiano
- Uboreshaji wa ubora wa matokeo kupitia marekebisho
- Ufuatiliaji (matokeo ya kati na ya mwisho yanahifadhiwa)

**Muundo wa Matokeo:**
- `question` - Swali la awali la mtumiaji
- `research` - Matokeo ya utafiti wa awali (pointi za ukweli)
- `final` - Muhtasari wa mwisho ulioboreshwa
- `models` - Ni mifano gani ilitumika kwa kila hatua

**Mawazo ya Kupanua:**
1. Ongeza wakala Mkosoaji kwa ukaguzi wa ubora
2. Tekeleza mawakala wa utafiti sambamba kwa vipengele tofauti
3. Ongeza wakala Mhakiki kwa uhakiki wa ukweli
4. Tumia mifano tofauti kwa viwango tofauti vya ugumu
5. Tekeleza mizunguko ya maoni kwa uboreshaji wa hatua kwa hatua


### Kiwango cha Juu: Usanidi wa Wakala Maalum

Jaribu kubadilisha tabia ya wakala kwa kurekebisha vigezo vya mazingira kabla ya kuendesha seli ya uanzishaji:

**Mifano Inayopatikana:**
- Tumia `foundry model ls` kwenye terminal kuona mifano yote inayopatikana
- Mifano: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, nk.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
